### `ising2d` Library Reference

The `ising2d` module provides a high-performance C++/CUDA implementation of the 2D Ising Model, exposed to Python via Pybind11.

#### 1. Initialization
Create a new simulation object.
* **`model = ising2d.IsingModel(L, T, J=1.0, h=0.0, seed=42)`**
    * `L`: Lattice side length (e.g., 1024 produces a $1024 \times 1024$ grid).
    * `T`: Temperature (Theoretical Critical Temp $T_c \approx 2.269$).
    * `J`: Interaction strength (default 1.0).
    * `h`: External magnetic field (default 0.0).
    * `seed`: Seed for the random number generator.

#### 2. Simulation & Physics
Evolve the system using the specified backend.
* **`model.update(mode, steps)`**
    * `steps`: Integer number of Monte Carlo steps.
    * `mode`: The backend execution strategy:
        * `ising2d.Mode.SERIAL`: Single-core CPU (Slowest, baseline).
        * `ising2d.Mode.OPENMP`: Multi-core CPU.
        * `ising2d.Mode.CUDA_GLOBAL`: GPU using Global Memory.
        * `ising2d.Mode.CUDA_SHARED`: GPU using Shared Memory (Fastest).
* **`model.device_synchronize()`**
    * **Crucial for Benchmarking:** Blocks Python execution until the GPU has finished all queued tasks. Without this, your timing code will be incorrect because CUDA launches kernels asynchronously.

#### 3. Data Access (Zero-Copy)
Access simulation data without copying memory buffers.
* **`model.lattice`**
    * Returns a **NumPy View** of the live spin grid.
    * **Read/Write:** Modifying this array in Python instantly updates the C++ memory.
    * **Format:** 2D array of integers (`-1` or `+1`).
* **`model.magnetization(mode)`**
    * Returns the average magnetization per spin $\langle m \rangle$.
* **`model.energy(mode)`**
    * Returns the total energy of the system $E$.



#### 4. Performance Tuning
Adjust hardware parameters on the fly.
* **`model.set_cuda_block_size(size)`**
    * Sets the CUDA thread block dimension (Valid: 8, 16, 32).
    * *Tip:* 16 or 32 usually gives the best performance on RTX cards.
* **`model.set_num_threads(n)`**
    * Sets the number of CPU threads used by the `OPENMP` mode.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import sys

project_path = "/home/william/MCP_project/Ising2D_with_CUDA"
if project_path not in sys.path:
    sys.path.append(project_path)

import ising2d


class visualize():
    def __init__(self, simulation):
        self.simulation = simulation
        self.L = simulation.lattice.shape[0]

    def plot_lattice(self, title='None', cmap='magma'):
        """
        Functions that plots the lattice configuration
        """
        plt.figure(figsize=(6, 6))
        plt.imshow(self.simulation.lattice, cmap=cmap)
        plt.title(title)
        plt.axis('off')
        plt.show()

    def animate(self, frames=100, steps_per_frame=50):
        """
        Functions that shows the animation until the reach of equilibrium of the spin lattice
        """
        fig, ax = plt.subplots(figsize=(6, 6))
        
        # Setup Initial Plot
        img = ax.imshow(self.simulation.lattice, cmap='magma', animated=True)
        title = ax.set_title("Init")
        ax.axis('off')

        c_mode = ising2d.Mode.SERIAL

        # 2. Update Loop
        def update(frame):
            if frame > 0:
                self.simulation.update(c_mode, steps_per_frame)
                
            # Update the image
            img.set_array(self.simulation.lattice)
            
            # Correct Step Label: Frame 0 * 10 = Step 0
            title.set_text(f"Step: {frame * steps_per_frame}")
            
            return img, title

        # Create Animation
        anim = FuncAnimation(fig, update, frames=frames, interval=50, blit=True)
        plt.close(fig)
        return HTML(anim.to_jshtml())

In [63]:
# 1. Setup Model (Small L is better for Serial, it's slow!)
model = ising2d.IsingModel(L=128, T=2.0, seed=42)

# 2. Attach Visualizer
viz = visualize(model)

# 3. Animate
viz.animate(frames=200, steps_per_frame=10)